# Module biogeme.expressions 

## Examples of use of each function

This webpage is for programmers who need examples of use of the functions of the module. The examples are designed to illustrate the syntax. They do not correspond to any meaningful model. For examples of models, visit  [biogeme.epfl.ch](http://biogeme.epfl.ch).

In [1]:
import datetime
print(datetime.datetime.now())

2020-04-29 10:33:58.032547


In [2]:
import biogeme.version as ver
print(ver.getText())

biogeme 3.2.6a [2020-04-29]
Version entirely written in Python
Home page: http://biogeme.epfl.ch
Submit questions to https://groups.google.com/d/forum/biogeme
Michel Bierlaire, Transport and Mobility Laboratory, Ecole Polytechnique Fédérale de Lausanne (EPFL)



In [3]:
import numpy as np
import pandas as pd

In [4]:
import biogeme.expressions as ex
import biogeme.database as db
import biogeme.models as models

We first create a small database

In [5]:
df = pd.DataFrame({'Person': [1, 1, 1, 2, 2],
                   'Exclude': [0, 0, 1, 0, 1],
                   'Variable1': [10, 20, 30, 40, 50],
                   'Variable2': [100, 200, 300, 400, 500],
                   'Choice': [1, 2, 3, 1, 2],
                   'Av1': [0, 1, 1, 1, 1],
                   'Av2': [1, 1, 1, 1, 1],
                   'Av3': [0, 1, 1, 1, 1]})
myData = db.Database('test', df)

The following type of expression is a literal called Variable that corresponds to an entry in the database.

In [6]:
Person = ex.Variable('Person')
Variable1 = ex.Variable('Variable1')
Variable2 = ex.Variable('Variable2')
Choice = ex.Variable('Choice')
Av1 = ex.Variable('Av1')
Av2 = ex.Variable('Av2')
Av3 = ex.Variable('Av3')

It is possible to add a new column to thre database, that creates a new variable that can be used in expressions.

In [7]:
newvar = ex.DefineVariable('newvar',
                           Variable1 + Variable2,
                           myData)
print(myData)

biogeme database test:
   Person  Exclude  Variable1  Variable2  Choice  Av1  Av2  Av3  newvar
0       1        0         10        100       1    0    1    0     110
1       1        0         20        200       2    1    1    1     220
2       1        1         30        300       3    1    1    1     330
3       2        0         40        400       1    1    1    1     440
4       2        1         50        500       2    1    1    1     550


The following type of expression is another literal, corresponding to an unknown parameter. 

In [8]:
beta1 = ex.Beta('beta1', 0, None, None, 0)
beta2 = ex.Beta('beta2', 0, None, None, 0)
beta3 = ex.Beta('beta3', 1, None, None, 1)
beta4 = ex.Beta('beta4', 0, None, None, 1)

Arithmetic operators are overloaded to allow standard manipulations of expressions. The first expression is $$e_1 = 2  \beta_1 - \frac{\exp(-\beta_2)}{\beta_3 (\beta_2 \geq \beta_1)},$$
where $(\beta_2 \geq \beta_1)$ equals 1 if $\beta_2 \geq \beta_1$ and 0 otherwise.

In [9]:
expr1 = 2 * beta1 - ex.exp(-beta2) / (beta3 * (beta2 >= beta1))
print(expr1)

((`2` * beta1(0)) - (exp((-beta2(0))) / (beta3(1) * (beta2(0) >= beta1(0)))))


The evaluation of expressions can be done in two ways. For simple expressions, the fonction getValue(), implemented in Python, returns the value of the expression.  

In [10]:
expr1.getValue()

-1.0

It is possible to modify the values of the parameters

In [11]:
newvalues = {'beta1': 1, 'beta2': 2, 'beta3': 3, 'beta4': 2}
expr1.changeInitValues(newvalues)
expr1.getValue()

1.954888238921129

The function getValue_c() is implemented in C++, and works for any expression. It requires a database as input, and evaluates the expression for each entry in the database.
In the following example, as no variable of the database is involved in the expression, the output of the expression is the same for each entry.

In [12]:
expr1.getValue_c(myData)

[1.954888238921129,
 1.954888238921129,
 1.954888238921129,
 1.954888238921129,
 1.954888238921129]

The following function scans the expression and extracts a dict with all free parameters.

In [13]:
expr1.setOfBetas()

{'beta1', 'beta2'}

Options can be set to extract free parameters, fixed parameters, or both. 

In [14]:
expr1.setOfBetas(free=False, fixed=True)

{'beta3'}

In [15]:
expr1.setOfBetas(free=True, fixed=True)

{'beta1', 'beta2', 'beta3'}

It is possible also to extract an elementary expression from its name.

In [16]:
expr1.getElementaryExpression('beta2')

beta2(2)

Let's consider an expression involving two variables $V_1$ and $V_2$: $$e_2 =2 \beta_1  V_1 - \frac{\exp(-\beta_2 V_2) }{ \beta_3  (\beta_2 \geq \beta_1)}.$$ Note that, in our example, the second term is numerically negligible with respect to the first one.

In [17]:
expr2 = 2 * beta1 * Variable1 - ex.exp(-beta2*Variable2) / (beta3 * (beta2 >= beta1))
print(expr2)

(((`2` * beta1(1)) * Variable1) - (exp(((-beta2(2)) * Variable2)) / (beta3(3) * (beta2(2) >= beta1(1)))))


It is not a simple expression anymore, and only the function getValue_c can be invoked.

In [18]:
expr2.getValue_c(myData)

[20.0, 40.0, 60.0, 80.0, 100.0]

The following function extracts the names of the parameters apprearing in the expression

In [19]:
expr2.setOfBetas(free=True,fixed=True)

{'beta1', 'beta2', 'beta3'}

The list of parameters can also be obtained in the form of a dictionary.

In [20]:
expr2.dictOfBetas(free=True,fixed=True)

{'beta1': beta1(1), 'beta2': beta2(2), 'beta3': beta3(3)}

The list of variables can also be obtained in the form of a dictionary

In [21]:
expr2.dictOfVariables()

{'Variable1': Variable1, 'Variable2': Variable2}

or a set...

In [22]:
expr2.setOfVariables()

{'Variable1', 'Variable2'}

Expressions are defined recursively, using a tree representation. The following function describes the type of the upper most node of the tree.

In [23]:
expr2.getClassName()

'Minus'

The signature is a formal representation of the expression, assigning identifiers to each node of the tree, and representing them starting from the leaves. It is easy to parse, and is passed to the C++ implementation. 

In [24]:
expr2.getSignature()

[b'<Numeric>{4634433904},2',
 b'<Beta>{4634280624}"beta1"[0],0,0',
 b'<Times>{4634433952}(2),4634433904,4634280624',
 b'<Variable>{4632122320}"Variable1",5,2',
 b'<Times>{4634433760}(2),4634433952,4632122320',
 b'<Beta>{4632075808}"beta2"[0],1,1',
 b'<UnaryMinus>{4634434000}(1),4632075808',
 b'<Variable>{4632122416}"Variable2",6,3',
 b'<Times>{4634434048}(2),4634434000,4632122416',
 b'<exp>{4634434096}(1),4634434048',
 b'<Beta>{4634280672}"beta3"[1],2,0',
 b'<Beta>{4632075808}"beta2"[0],1,1',
 b'<Beta>{4634280624}"beta1"[0],0,0',
 b'<GreaterOrEqual>{4634434192}(2),4632075808,4634280624',
 b'<Times>{4634434144}(2),4634280672,4634434192',
 b'<Divide>{4634434240}(2),4634434096,4634434144',
 b'<Minus>{4634434288}(2),4634433760,4634434240']

The elementary expressions are
- free parameters,
- fixed parameters,
- random variables (for numerical integration),
- draws (for Monte-Carlo integration), and
- variables from the database.

The following function extracts all elementary expressions from a list of formulas, give them a unique numbering, and return them organized by group, as defined above (with the exception of the variables, that are directly available in the database).

In [25]:
collectionOfFormulas = [expr1, expr2]
(elementaryExpressionIndex,
    allFreeBetas,freeBetaNames,
    allFixedBetas,
    fixedBetaNames,
    allRandomVariables,
    randomVariableNames,
    allDraws,
    drawNames) =\
ex.defineNumberingOfElementaryExpressions(collectionOfFormulas,
                                         list(myData.data.columns))

Unique numbering for all elementary expressions

In [26]:
elementaryExpressionIndex

{'beta1': 0,
 'beta2': 1,
 'beta3': 2,
 'Person': 3,
 'Exclude': 4,
 'Variable1': 5,
 'Variable2': 6,
 'Choice': 7,
 'Av1': 8,
 'Av2': 9,
 'Av3': 10,
 'newvar': 11}

In [27]:
allFreeBetas

{'beta1': beta1(1), 'beta2': beta2(2)}

Each elementary expression has two ids. One unique across all elementary expressions, and one unique within each specific group

In [28]:
[(i.uniqueId, i.betaId) for k, i in allFreeBetas.items()]

[(0, 0), (1, 1)]

In [29]:
freeBetaNames

['beta1', 'beta2']

In [30]:
allFixedBetas

{'beta3': beta3(3)}

In [31]:
[(i.uniqueId, i.betaId) for k, i in allFixedBetas.items()]

[(2, 0)]

In [32]:
fixedBetaNames

['beta3']

In [33]:
allRandomVariables

{}

Monte Carlo integration is based on draws. 

In [34]:
myDraws = ex.bioDraws('myDraws', 'UNIFORM')
expr3 = ex.MonteCarlo(myDraws * myDraws)

In [35]:
print(expr3)

MonteCarlo((bioDraws("myDraws", "UNIFORM") * bioDraws("myDraws", "UNIFORM")))


Note that draws are not random variables, used for numerical integration.

In [36]:
expr3.dictOfRandomVariables()

{}

The following function reports the draws involved in an expression.

In [37]:
expr3.dictOfDraws()

{'myDraws': 'UNIFORM'}

The expression is a Monte-Carlo integration.

In [38]:
expr3.getClassName()

'MonteCarlo'

Here is its value. It is an approximation of $\int_0^1 x^2 dx=\frac{1}{3}$.

In [39]:
expr3.getValue_c(myData, numberOfDraws=100000)

[0.33180323425839536,
 0.3331305583559927,
 0.3327955183329525,
 0.33312697598824154,
 0.33306330921886285]

Here is its signature.

In [40]:
expr3.getSignature()

[b'<bioDraws>{4634563488}"myDraws",0,0',
 b'<bioDraws>{4634563488}"myDraws",0,0',
 b'<Times>{4634563392}(2),4634563488,4634563488',
 b'<MonteCarlo>{4634563440}(1),4634563392']

The same integral can be calculated using numerical integration, declaring a random variable. 

In [41]:
omega = ex.RandomVariable('omega')

Numerical integration calculates integrals between $-\infty$ and $+\infty$. Here, the interval being $[0,1]$, a change of variables is required.

In [42]:
a = 0
b = 1
x = a + (b - a) / ( 1 + ex.exp(-omega))
dx = (b - a) * ex.exp(-omega) * (1 + ex.exp(-omega))**(-2) 
integrand = x * x
expr4 = ex.Integrate(integrand * dx /(b - a), 'omega')

In this case, omega is a random variable.

In [43]:
expr4.dictOfRandomVariables()

{'omega': omega}

In [44]:
print(expr4)

Integrate(((((`0` + (`1` / (`1` + exp((-omega))))) * (`0` + (`1` / (`1` + exp((-omega)))))) * ((`1` * exp((-omega))) * ((`1` + exp((-omega))) ** `-2`))) / `1`), "omega")


Calculating its value requires the C++ implementation.

In [45]:
expr4.getValue_c(myData)

[0.3333323120662823,
 0.3333323120662823,
 0.3333323120662823,
 0.3333323120662823,
 0.3333323120662823]

We illustrate now the Elem function. It takes two arguments: a dictionary, and a formula for the key. For each entry in the database, the formula is evaluated, and its result identifies which formula in the dictionary should be evaluated.
Here is 'Person' is 1, the expression is $$e_1=2  \beta_1 - \frac{\exp(-\beta_2)}{\beta_3 (\beta_2 \geq \beta_1)},$$ and if 'Person' is 2, the expression is $$e_2=2 \beta_1  V_1 - \frac{\exp(-\beta_2 V_2) }{ \beta_3  (\beta_2 \geq \beta_1)}.$$ As it is a regular expression, it can be included in any formula. Here, we illustrate it by dividing the result by 10.

In [46]:
elemExpr = ex.Elem({1: expr1, 2: expr2}, Person) 
expr5 =  elemExpr / 10
print(expr5)

({{1:((`2` * beta1(1)) - (exp((-beta2(2))) / (beta3(3) * (beta2(2) >= beta1(1))))), 2:(((`2` * beta1(1)) * Variable1) - (exp(((-beta2(2)) * Variable2)) / (beta3(3) * (beta2(2) >= beta1(1)))))}[Person] / `10`)


In [47]:
expr5.dictOfVariables()

{'Variable1': Variable1, 'Variable2': Variable2, 'Person': Person}

In [48]:
expr5.getValue_c(myData)

[0.19548882389211292, 0.19548882389211292, 0.19548882389211292, 8.0, 10.0]

The next expression is simply the sum of multiples expressions. The argument is a list of expressions. 

In [49]:
expr6 = ex.bioMultSum([expr1, expr2, expr4])

In [50]:
print(expr6)

bioMultSum(((`2` * beta1(1)) - (exp((-beta2(2))) / (beta3(3) * (beta2(2) >= beta1(1))))) + (((`2` * beta1(1)) * Variable1) - (exp(((-beta2(2)) * Variable2)) / (beta3(3) * (beta2(2) >= beta1(1))))) + Integrate(((((`0` + (`1` / (`1` + exp((-omega))))) * (`0` + (`1` / (`1` + exp((-omega)))))) * ((`1` * exp((-omega))) * ((`1` + exp((-omega))) ** `-2`))) / `1`), "omega"))


In [51]:
expr6.getValue_c(myData, 100000)

[22.28822055098741,
 42.28822055098741,
 62.28822055098741,
 82.2882205509874,
 102.2882205509874]

We now illustrate how to calculate a logit model, that is $$ \frac{y_1 e^{V_1}}{y_0 e^{V_0}+y_1 e^{V_1}+y_2 e^{V_2}}$$ where $V_0=-\beta_1$, $V_1=-\beta_2$ and $V_2=-\beta_1$, and $y_i = 1$, $i=1,2,3$.

In [52]:
V = {0: -beta1, 1: -beta2, 2: -beta1}
av = {0: 1, 1: 1, 2: 1}
expr7 = ex.LogLogit(V, av, 1)

In [53]:
expr7.getValue()

-1.861994804058251

It is actually better to use the C++ implementation, available in the module models

In [54]:
expr8 = models.loglogit(V,av,1)

In [55]:
expr8.getValue_c(myData)

[-1.8619948040582512,
 -1.8619948040582512,
 -1.8619948040582512,
 -1.8619948040582512,
 -1.8619948040582512]

As the result is a numpy array, it can be used for any calculation. Here, we show how to calculate the logsum

In [56]:
for v in V.values():
    print(v.getValue_c(myData))

[-1.0, -1.0, -1.0, -1.0, -1.0]
[-2.0, -2.0, -2.0, -2.0, -2.0]
[-1.0, -1.0, -1.0, -1.0, -1.0]


In [57]:
logsum = np.log(np.sum([np.exp(v.getValue_c(myData)) 
                        for v in V.values()], axis=1))
logsum

array([ 0.60943791, -0.39056209,  0.60943791])

It is possible to calculate the derivative of a formula with respect to a literal: $$e_9=\frac{\partial e_8}{\partial \beta_2}.$$

In [58]:
expr9 = ex.Derive(expr8, 'beta2')

In [59]:
expr9.getValue_c(myData)

[-0.8446375965030364,
 -0.8446375965030364,
 -0.8446375965030364,
 -0.8446375965030364,
 -0.8446375965030364]

Biogeme also provides an approximation of the CDF of the normal distribution: $$e_{10}= \frac{1}{{\sigma \sqrt {2\pi } }}\int_{-\infty}^t e^{{{ - \left( {x - \mu } \right)^2 } \mathord{\left/ {\vphantom {{ - \left( {x - \mu } \right)^2 } {2\sigma ^2 }}} \right. } {2\sigma ^2 }}}dx$$

In [60]:
expr10 = ex.bioNormalCdf(Variable1 / 10 - 1)

In [61]:
expr10.getValue_c(myData)

[0.5,
 0.8413447460685283,
 0.9772498680518218,
 0.99865010196837,
 0.9999683287581669]

Min and max operators are also available. To avoid any ambiguity with the Python operator, they are called bioMin and bioMax. 

In [62]:
expr11 = ex.bioMin(expr5, expr10)
expr11.getValue_c(myData)

[0.19548882389211292,
 0.19548882389211292,
 0.19548882389211292,
 0.99865010196837,
 0.9999683287581669]

In [63]:
expr12 = ex.bioMax(expr5, expr10)
expr12.getValue_c(myData)

[0.5, 0.8413447460685283, 0.9772498680518218, 8.0, 10.0]

For the sake of efficiency, it is possible to specify explicitly a linear function, where each term is the product of a parameter and a variable.

In [64]:
terms = [(beta1, ex.Variable('Variable1')),
         (beta2, ex.Variable('Variable2')),
         (beta3, ex.Variable('newvar'))]

In [65]:
expr13 = ex.bioLinearUtility(terms)

In [66]:
expr13.getValue_c(myData)

[540.0, 1080.0, 1620.0, 2160.0, 2700.0]

In terms of specification, it is equivalent to the expression below. But the calculation of the derivates is more efficient, as the linear structure of the specification is exploited.

In [67]:
expr13bis = beta1 * Variable1 + beta2 * Variable2 + beta3 * newvar

In [68]:
expr13bis.getValue_c(myData)

[540.0, 1080.0, 1620.0, 2160.0, 2700.0]

A Pythonic way to write a linear utility function

In [69]:
variables = ['v1', 'v2', 'v3', 'cost', 'time', 'headway']
coefficients = {f'{v}': ex.Beta(f'beta_{v}', 0, None, None, 0) 
                for v in variables}
terms = [coefficients[v] * ex.Variable(v) for v in variables]
util = sum(terms)
print(util)

((((((`0` + (beta_v1(0) * v1)) + (beta_v2(0) * v2)) + (beta_v3(0) * v3)) + (beta_cost(0) * cost)) + (beta_time(0) * time)) + (beta_headway(0) * headway))


# Signatures

The Python library communicates the expressions to the C++ library using a syntax called a "signature". We describe and illustrate now the signature for each expression. Each expression is identified by an identifier provided by Python using the function 'id'. 

In [70]:
id(expr1)

4634280192

## Numerical expression

&lt;Numeric&gt;{identifier},value

In [71]:
ex.Numeric(0).getSignature()

[b'<Numeric>{4634696240},0']

## Beta parameters

&lt;Beta&gt;{identifier}"name"[status],uniqueId,betaId'
where 
- status is 0 for free parameters, and non zero for fixed parameters,
- uniqueId is a unique index given by Biogeme to all elementary expressions,
- betaId is a unique index given by Biogeme to all free parameters, and to all fixed parameters.

In [72]:
beta1.getSignature()

[b'<Beta>{4634280624}"beta1"[0],0,0']

In [73]:
beta3.getSignature()

[b'<Beta>{4634280672}"beta3"[1],2,0']

## Variables

&lt;Variable&gt;{identifier}"name",uniqueId,variableId 
where
- uniqueId is a unique index given by Biogeme to all elementary expressions,
- variableId is a unique index given by Biogeme to all variables.


In [74]:
Variable1.getSignature()

[b'<Variable>{4632122320}"Variable1",5,2']

## Random variables

&lt;RandomVariable&gt;{identifier}"name",uniqueId,randomVariableId
where
- uniqueId is a unique index given by Biogeme to all elementary expressions,
- randomVariableId is a unique index given by Biogeme to all random variables.

In [75]:
omega.getSignature()

[b'<RandomVariable>{4634589264}"omega",3,0']

## Draws

&lt;bioDraws&gt;{identifier}"name",uniqueId,drawId
where
- uniqueId is a unique index given by Biogeme to all elementary expressions,
- drawId is a unique index given by Biogeme to all draws.


In [76]:
myDraws.getSignature()

[b'<bioDraws>{4634563488}"myDraws",0,0']

## General expression

&lt;operator&gt;{identifier}(numberOfChildren),idFirstChild,idSecondChild,idThirdChild, etc...
where the number of identifiers given after the comma matches the reported number of children. 

Specific examples are reported below.

### Binary operator

&lt;operator&gt;{identifier}(2),idFirstChild,idSecondChild 
where operator is one of: 
    - 'Plus'
    - 'Minus'
    - 'Times'
    - 'Divide'
    - 'Power'
    - 'bioMin'
    - 'bioMax'
    - 'And'
    - 'Or'
    - 'Equal'
    - 'NotEqual'
    - 'LessOrEqual'
    - 'GreaterOrEqual'
    - 'Less'
    - 'Greater'


In [77]:
sum = beta1 + Variable1

In [78]:
sum.getSignature()

[b'<Beta>{4634280624}"beta1"[0],0,0',
 b'<Variable>{4632122320}"Variable1",5,2',
 b'<Plus>{4634652576}(2),4634280624,4632122320']

### Unary operator

&lt;operator&gt;{identifier}(1),idChild, 
where operator is one of: 
    - 'UnaryMinus'
    - 'MonteCarlo'
    - 'bioNormalCdf'
    - 'PanelLikelihoodTrajectory'
    - 'exp'
    - 'log'

In [79]:
m = -beta1

In [80]:
m.getSignature()

[b'<Beta>{4634280624}"beta1"[0],0,0',
 b'<UnaryMinus>{4634697536}(1),4634280624']

## LogLogit

&lt;LogLogit&gt;{identifier}(nbrOfAlternatives),chosenAlt,altNumber,utility,availability,altNumber,utility,availability, etc.

In [81]:
expr7.getSignature()

[b'<Numeric>{4634650944},1',
 b'<Beta>{4634280624}"beta1"[0],0,0',
 b'<UnaryMinus>{4634651328}(1),4634280624',
 b'<Beta>{4632075808}"beta2"[0],1,1',
 b'<UnaryMinus>{4634650176}(1),4632075808',
 b'<Beta>{4634280624}"beta1"[0],0,0',
 b'<UnaryMinus>{4634651376}(1),4634280624',
 b'<Numeric>{4634651472},1',
 b'<Numeric>{4634651520},1',
 b'<Numeric>{4634651568},1',
 b'<LogLogit>{4634651424}(3),4634650944,0,4634651328,4634651472,1,4634650176,4634651520,2,4634651376,4634651568']

## Derive

&lt;Derive&gt;{identifier},id of expression to derive,unique index of elementary expression

In [82]:
expr9.getSignature()

[b'<Numeric>{4634652528},1',
 b'<Beta>{4634280624}"beta1"[0],0,0',
 b'<UnaryMinus>{4634651328}(1),4634280624',
 b'<Beta>{4632075808}"beta2"[0],1,1',
 b'<UnaryMinus>{4634650176}(1),4632075808',
 b'<Beta>{4634280624}"beta1"[0],0,0',
 b'<UnaryMinus>{4634651376}(1),4634280624',
 b'<Numeric>{4634650848},1',
 b'<Numeric>{4634652624},1',
 b'<Numeric>{4634652432},1',
 b'<_bioLogLogit>{4634650560}(3),4634652528,0,4634651328,4634650848,1,4634650176,4634652624,2,4634651376,4634652432',
 b'<Derive>{4634694848},4634650560,1']

## Integrate

&lt;Integrate&gt;{identifier},id of expression to derive,index of random variable

In [83]:
expr4.getSignature()

[b'<Numeric>{4634605840},0',
 b'<Numeric>{4634605600},1',
 b'<Numeric>{4634605312},1',
 b'<RandomVariable>{4634589264}"omega",3,0',
 b'<UnaryMinus>{4634605360}(1),4634589264',
 b'<exp>{4634605744}(1),4634605360',
 b'<Plus>{4634605408}(2),4634605312,4634605744',
 b'<Divide>{4634605504}(2),4634605600,4634605408',
 b'<Plus>{4634605792}(2),4634605840,4634605504',
 b'<Numeric>{4634605840},0',
 b'<Numeric>{4634605600},1',
 b'<Numeric>{4634605312},1',
 b'<RandomVariable>{4634589264}"omega",3,0',
 b'<UnaryMinus>{4634605360}(1),4634589264',
 b'<exp>{4634605744}(1),4634605360',
 b'<Plus>{4634605408}(2),4634605312,4634605744',
 b'<Divide>{4634605504}(2),4634605600,4634605408',
 b'<Plus>{4634605792}(2),4634605840,4634605504',
 b'<Times>{4634605888}(2),4634605792,4634605792',
 b'<Numeric>{4634606032},1',
 b'<RandomVariable>{4634589264}"omega",3,0',
 b'<UnaryMinus>{4634605264}(1),4634589264',
 b'<exp>{4634605648}(1),4634605264',
 b'<Times>{4634605984}(2),4634606032,4634605648',
 b'<Numeric>{46346062

## Elem

&lt;Elem&gt;{identifier}(numberOfExpressions),keyId,value1,expression1,value2,expression2, etc...

where
- keyId is the identifier of the expression calculating the key,
- the number of pairs valuex,expressionx must correspond to the value of numberOfExpressions

In [84]:
elemExpr.getSignature()

[b'<Variable>{4632121552}"Person",3,0',
 b'<Numeric>{4634280384},2',
 b'<Beta>{4634280624}"beta1"[0],0,0',
 b'<Times>{4634082080}(2),4634280384,4634280624',
 b'<Beta>{4632075808}"beta2"[0],1,1',
 b'<UnaryMinus>{4634280096}(1),4632075808',
 b'<exp>{4634280240}(1),4634280096',
 b'<Beta>{4634280672}"beta3"[1],2,0',
 b'<Beta>{4632075808}"beta2"[0],1,1',
 b'<Beta>{4634280624}"beta1"[0],0,0',
 b'<GreaterOrEqual>{4634280288}(2),4632075808,4634280624',
 b'<Times>{4634280000}(2),4634280672,4634280288',
 b'<Divide>{4634280336}(2),4634280240,4634280000',
 b'<Minus>{4634280192}(2),4634082080,4634280336',
 b'<Numeric>{4634433904},2',
 b'<Beta>{4634280624}"beta1"[0],0,0',
 b'<Times>{4634433952}(2),4634433904,4634280624',
 b'<Variable>{4632122320}"Variable1",5,2',
 b'<Times>{4634433760}(2),4634433952,4632122320',
 b'<Beta>{4632075808}"beta2"[0],1,1',
 b'<UnaryMinus>{4634434000}(1),4632075808',
 b'<Variable>{4632122416}"Variable2",6,3',
 b'<Times>{4634434048}(2),4634434000,4632122416',
 b'<exp>{463443

## bioLinearUtility

&lt;bioLinearUtility&gt;{identifier}(numberOfTerms), beta1_exprId, beta1_uniqueId, beta1_name, variable1_exprId, variable1_uniqueId, variable1_name, etc...

where 6 entries are provided for each term:
    - beta1_exprId is the expression id of the beta parameter
    - beta1_uniqueId is the unique id of the beta parameter
    - beta1_name is the name of the parameter
    - variable1_exprId is the expression id of the variable
    - variable1_uniqueId is the unique id of the variable
    - variable1_name is the name of the variable


In [85]:
expr13.getSignature()

[b'<Beta>{4634280624}"beta1"[0],0,0',
 b'<Beta>{4632075808}"beta2"[0],1,1',
 b'<Beta>{4634280672}"beta3"[1],2,0',
 b'<Variable>{4634739280}"Variable1",5,2',
 b'<Variable>{4634740528}"Variable2",6,3',
 b'<Variable>{4634739568}"newvar",11,8',
 b'<bioLinearUtility>{4634740096}(3),4634280624,0,beta1,4634739280,5,Variable1,4632075808,1,beta2,4634740528,6,Variable2,4634280672,2,beta3,4634739568,11,newvar']